# Introducción al PLN con Jupyter y Python
### NLTK, WordNet, Word2Vec y similitud
José A. Troyano &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 27/02/2018

## Contenido
* [1. Lectura y tokenización](#sec_lectura)
* [2. Cálculo de frecuencias y transformaciones básicas](#sec_frec)
* [3. Etiquetado POS](#sec_pos)
* [4. WordNet](#sec_wordnet)
* [5. Word2vec](#sec_wor2vec)
* [6. Similitud de textos](#sec_similitud)

En este _notebook_ se recogen una serie de ejemplos de uso de distintas herramientas implementadas en <code>Python</code> <code>3</code> para el Procesamiento del Lenguaje Natural. 

El formato _notebook_ tiene la ventaja, y versatilidad, de integrar tanto explicaciones como fragmentos de programa en un mismo recurso. La unidad básica de un _notebook_ es la _celda_, que puede ser de dos tipos: _de código_ y _markdown_. Las celdas de código contienen trozos de programa que podemos ejecutar, mientras que las celdas _markdown_ contienen explicaciones (esta celda que estás leyendo es una celda _markdown_). Este diseño convierte a los _notebooks_ en un instrumento muy potente para el trabajo científico: se puede usar como _cuaderno de bitácora_ para guardar de forma organizada nuestros experimentos, o también como medio para compartir estos experimentos y que puedan ser reproducidos por otras personas.

Otra ventaja de los _notebooks_ es que la plataforma de trabajo no es excesivamente compleja. Usarlos es muy intuitivo y crearlos no es nada complicado. Las opciones de la _interfaz web_ son bastante simples y (con conocimientos muy básicos de <code>LaTex</code>, <code>HTML</code> y <code>Markdown</code>) podemos empezar a crear nuestros propios _notebooks_ casi desde el minuto cero. Además, basta hacer doble _click_ sobre cualquier celda para ver _cómo está hecha_ y aprender de otros.

Los _notebooks_ de <code>Jupyter</code> pueden soportar muchos lenguajes de programación. En este usaremos <code>Python</code> que, de hecho, fue el lenguaje en el que se basaba el proyecto antecesor de <code>Jupyter</code> (denominado <code>IPython</code>). Pero esta no es la razón por la que se ha elegido <code>Python</code>. Las verdaderas razones (y de bastante peso) son:
- <code>Python</code> es un lenguaje muy intuitivo y con una curva de aprendizaje suave. Exige escribir lo justo, y es un lenguaje idóneo para que personas que no tengan formación tecnológica puedan iniciarse en la programación.
- Hay una comunidad de desarrolladores inmensa en <code>Python</code> en multitud de ámbitos. En concreto, para la comunidad PLN hay desarrollados muchos paquetes que dan soporte a muchas tareas del Procesamiento del Lenguaje Natural.

Este _notebook_ está diseñado para que pueda ser usado por personas sin conocimientos de programación. Los fragmentos de _código_ (programas en la jerga informática) son simples y están comentados para que pueda comprenderse en todo momento qué se está haciendo. Lo recomendable es usar este _notebook_ en dos fases:
- Fase de aprendizaje: ejecutar las celdas, comprender las explicaciones y comentarios, y corroborar con las salidas que comprendenmos lo que _está pasando_.
- Fase de pruebas propias: hacer una copia del _notebook_ y cambiar (sin miedo) todo lo que se nos ocurra. Es el momento de empezar a experimentar con ideas propias y aprender al mismo tiempo.

Todas los paquetes que vamos a usar están disponibles en la distribución de <code>Anaconda</code> que incluye, además, un intérprete de <code>Python</code> y el propio <code>Jupyter</code>. De modo que lo más cómodo, y recomendable, es instalar <code>Anaconda</code>
(https://conda.io/docs/user-guide/install/download.html).

Una de las herramientas que más usaremos será <code>NLTK</code>. Para empezar a usarla lo único que necesitamos hacer es _importar_ el paquete <code>nltk</code>:

In [ ]:
import nltk

Cuando se instala <code>NLTK</code> (si hemos instalado <code>Anaconda</code> ya estará accesible para nuestro intérprete de <code>Python</code>), por razones de espacio no se descargan todos los recursos integrados en él (corpus, lexicones, ...). Para descargar un recurso debemos ejecutar la función <code>download()</code> indicando el nombre del recurso. Si queremos descargar los más populares basta con ejecutar: 

In [ ]:
nltk.download('popular')

## 1. Lectura y tokenización <a name="sec_lectura"/>

En esta primera sección usaremos <code>NLTK</code> para realizar algunas tareas básicas de tratamientos de textos, como la lectura de corpus, segmentación en párrafos, frases o palabras. Empezaremos a trabajar con un fichero que contiene El Quijote. El texto completo en formato <code>txt</code> se puede obtener desde el proyecto Gutemberg:  https://www.gutenberg.org/files/

El fichero <code>el-quijote.txt</code> debe encontrarse en una carpeta llamada <code>datos</code> que, a su vez, se encuentra en la misma carpeta en la que estamos ejecutando el _notebook_. Con las siguientes instrucciones <code>Python</code> leeremos todo el contenido del fichero y lo almacenaremos en una variable llamada <code>texto</code>:

In [ ]:
# Lectura del fichero 'el-quijote.txt' y carga en variable 'texto' 
fichero = open('./datos/el-quijote.txt', encoding='utf-8')     # Abrimos el fichero
texto = fichero.read()                                         # Leemos todo el contenido
fichero.close()                                                # Cerramos el fichero

Desde el momento en el que ejecutemos la celda anterior ya tenemos disponible la variable <code>texto</code>. La variable <code>texto</code> es de tipo _cadena de caracteres_ (<code>str</code> es la palabra reservada en <code>Python</code> para ese tipo de datos). En la siguiente celda de código podemos ver parte de su contenido:

In [ ]:
# Test de la lectura
print(texto[:500])               # Solo mostramos los primeros 500 caracteres de El Quijote

Llamaremos celdas de _test_ a las celdas como la anterior. Son celdas de código que no sirven para definir cosas (y que por tanto no podemos considerar que estrictamente forman parte de la solución que estamos construyendo), pero que nos ayudan a comprobar que vamos por buen camino. 

Será bastante normal que a lo largo del _notebook_ las celdas de código vengan en parejas: una para resolver parte de un problema y la siguiente para ejecutar un test. Comenzaremos estas celdas de test con el comentario <code># Test de ...</code> 

Lo primero que necesitaremos hacer con un texto tan grande como el de El Quijote es empezar a estructurarlo (en párrafos, frases, palabras, ...). La siguiente celda muestra cómo realizar la separación en párrafos. Se entiende que empieza un nuevo párrafo cuando aparecen dos _saltos de línea_ (carácter <code>'\n'</code>) seguidos. El grueso del trabajo lo hace la función <code>split</code> (es un método de las variables tipo <code>str</code>) que, aplicada a una variable que contiene un texto, la divide en trozos según la secuencia de caracteres que se le indica:

In [ ]:
# Segmentación en párrafos
parrafos = texto.split('\n\n')           # texto.split('\n\n') produce una lista de de 'str'

In [ ]:
# Test de la segmentación en párrafos
for parrafo in parrafos[:4]:             # Para cada párrafo (solo los cuatro primeros)
    print('[PÁRRAFO]')                   # Mostramos la etiqueta '[PÁRRAFO]'
    print(parrafo, '\n')                 # Mostramos el contenido del párrafo y dejamos una línea en blanco

Para separar un párrafo en frases podemos usar la función <code>sent_tokenize</code> disponible en <code>NLTK</code>. Esta función recibe el texto a segmentar, y el idioma del texto para que tenga en cuenta los símbolos de puntuación pertinentes:

In [ ]:
# Segmentación en frases
frases = []                                                       # Lista de frases inicialmente vacía
for parrafo in parrafos:                                          # Para cada párrafo
    parrafo = parrafo.replace('\n', ' ')                          # Cambiar los '\n' por ' '
    frases_p = nltk.sent_tokenize(parrafo, language='spanish')    # Método de NLTK para separar el párrafo en frases
    frases = frases + frases_p                                    # Añadir las frases del párrafo a la lista final de frases

In [ ]:
# Test de la segmentación en frases
for frase in frases[4:10]:                       # Para cada frase (solo de la 5 a la 9)
    print('[FRASE]', frase, '\n')                # Mostramos la frase y dejamos una línea en blanco

Separar una frase en palabras es también muy simple gracias a <code>NLTK</code>. En este caso usaremos la función <code>word_tokenize</code> a la que también podemos indicarle el idioma a través del parámetro <code>language</code>:

In [ ]:
# Tokenización
palabras = []                                                        # Lista de palabras inicialmente vacía
for frase in frases:                                                 # Para cada frase
    palabras_frase = nltk.word_tokenize(frase, language='spanish')   # Método de NLTK para separar la frase en palabras
    palabras = palabras + palabras_frase                             # Añadir las palabras de la frase a la lista final

In [ ]:
# Test de la tokenización
print(palabras[:100])

## 2. Cálculo de frecuencias y transformaciones básicas <a name="sec_frec"/>

Con las operaciones de la sección anterior ya hemos dado un primer paso en el análisis de textos escritos en lenguaje natural: hemos empezado dotar de cierta estructura a los textos gracias a las distintas segmentaciones que hemos aplicado (en párrafos, frases y palabras). El concepto de _estructura_ es clave a la hora de procesar información como la que se contiene en los textos. De hecho, el término _información no estructurada_ se usa para denominar a toda aquella información (como los textos, imágenes, audio, vídeo, ...) que es fácilmente interpretable por los humanos pero que no lo son tanto para los ordenadores. Una parte importante del trabajo a realizar para procesar información no estructurada es encontrar métodos que obtengan a partir de esta información modelos estructurados sobre los que trabajar.

En esta segunda sección vamos a empezar a aprovechar la estructura que ya tenemos para extraer algunas informaciones relevantes de los textos. Y también aprenderemos a filtrar aquellas informaciones que (para la resolución de ciertas tareas) no nos van a resultar útiles.

Lo primero que haremos será contar palabras. <code>NLTK</code> nos ofrece una función que nos resuelve directamente esa tarea. Se trata de <code>FreqDist</code> que toma una lista de palabras y construye un objeto de tipo <code>FreqDist</code>. Un objeto es una unidad de información que algutina datos (variables propias del objeto) y comportamiento (funciones propias del objeto llamadas _métodos_). Ya hemos usado objetos en este _notebook_, por ejemplo todas las variables de tipo _cadena_ o _lista_ que hemos usado son objetos.

En la siguiente celda construiremos un objeto <code>FreqDist</code> que mantiene los datos correspondientes a la distribución de las frecuencias de todas las palabras de la lista que recibe la función:

In [ ]:
# Cálculo de la frecuencia de las palabras
freq = nltk.FreqDist(palabras)

Y en la siguiente celda de test, mostraremos el objeto (solo un resumen) y usaremos su método <code>plot</code> para visualizar la distribución con un gráfico.

In [ ]:
# Test del cálculo de frecuencias
print(freq)                                # Muestra la información básica de un objeto FreqDist
freq.plot(25, cumulative=False)            # Genera una gráfica a partir del objeto FreqDist

Una de las operaciones básicas para ciertas tareas PLN, como la clasificación automática de documentos, es la eliminación de palabras que no resultan útiles para resolver dichas tareas. A esas palabras se les suele denominar _huecas_ (_stopwords_ en inglés) y suelen ser palabras funcionales que, en cierto modo, no están cargadas de contenido.

<code>NLTK</code> incluye listados de palabras huecas en varios idiomas. Solo con eso, y con una característica del lenguaje <code>Python</code> denominada _listas por comprensión_, es muy fácil eliminar palabras huecas de una lista de palabras.

Las listas por comprensión son una manera muy compacta de crear listas, generalmente a partir de otras listas. Al ser una notación compacta, de primeras puede parecer un poco críptica. Pero cuando nos acostumbramos a su uso, encontraremos que es elgo muy natural y, sobre todo, muy potente. Está inspirada en la forma en la que en matemáticas se definen conjuntos. Por ejemplo para definir el conjunto de todos los números pares y múltiplos de 3 podríamos usar esta descripción:
$$
numeros = \{3x \;\;|\;\; x \in \mathbb{N} \;\wedge\; 2(x/2) = x\}
$$

En esta descripción hay un conjunto generador $\mathbb{N}$, una condición $2(x/2) = x$ y una expresión constructora ($3x$). Precisamente esos tres elementos serán los que usemos en la definción de la lista por comprensión de un conjunto similar  en <code>Python</code> (no generaremos infinitos números, solo los menores que 300). Ahora esos elementos no serán fórmulas matemáticas, sino trozos de código escritos en <code>Python</code>:
- Lista generadora: <code>x in range(1,100)</code>
- Condición: <code>x==(int(x/2)*2)</code>
- Expresión constructora: <code>3*x</code>

In [ ]:
numeros = [3*x for x in range(1, 100) if x==(int(x/2)*2)]
print(numeros)

Utilizaremos una idea similar para definir la lista por comprensión que nos permitirá eliminar las palabras huecas. Los tres elementos de nuestra definición son:
- Lista generadora: <code>p in palabras</code>
- Condición: <code>p not in stopwords_esp</code>
- Expresión constructora: <code>p</code>

In [ ]:
# Eliminación de stopwords
stopwords_esp = nltk.corpus.stopwords.words('spanish')               # Cargamos las palabras huecas en una lista
palabras_ns = [p for p in palabras if p not in stopwords_esp]        # Eliminamos las palabras huecas por comprensión 

In [ ]:
# Test de eliminación de stopwords
print(palabras_ns[:100])
print(stopwords_esp)

Podemos repetir el cálculo de frecuencias, ahora con la lista de palabras que no contienen _stopwords_:

In [ ]:
# Cálculo de la frecuencia de las palabras no huecas
freq_ns = nltk.FreqDist(palabras_ns)

In [ ]:
# Test del cálculo de frecuencias de palabras no huecas
freq_ns.plot(25, cumulative=False)

Y en la siguiente celda tenemos un procesamiento un poco más completo que inlcuye los siguientes pasos:
- Eliminación de símbolos de puntuación
- Conversión de todas las palabras a minúsculas
- Eliminación de palabras huecas (aprovechamos para, además, mostrar cómo ampliar la lista de palabras huecas que nos proporciona <code>NLTK</code>)

In [ ]:
# Eliminación de símbolos no alfabéticos (de puntuación)
palabras_np = [p for p in palabras if p.isalpha()]

# Conversión a minúsculas
palabras_np_lw = [p.lower() for p in palabras_np]

# Ampliación de las stopwords
mis_stopwords = stopwords_esp + ['si', 'tan']

# Eliminación de stopwords
palabras_np_lw_ns = [p for p in palabras_np_lw if p not in mis_stopwords]

# Cálculo de las frecuencias a partir de esta nueva secuencia de palabras
freq_np_lw_ns = nltk.FreqDist(palabras_np_lw_ns)

In [ ]:
# Test del cálculo de frecuencias de palabras no-puntuación/minúsculas/no-huecas
freq_np_lw_ns.plot(25, cumulative=False)

## 3. Etiquetado POS <a name="sec_pos"/>

Las transformaciones que hemos visto en la sección anterior están pensadas para mantener solo aquellas palabras con mayor _carga semántica_ y son útiles solo para algunas tareas PLN. Una de ellas es la clasificación automática de textos, en la que _nos podemos permitir el lujo_ de despreciar algunas palabras o símbolos y, aún así, obtener buenos resultados de clasificación. Esto se debe a que, en la mayoría de situaciones, basta con analizar las palabras _más importantes_ de un texto para determinar a qué categoría pertenece. En este tipo de tareas se usa el deominado modelo BOW (_Bag Of Words_) que representa un documento mediante un vector (de tantas dimensiones como el vocabulario de palabras de nuestro dominio, por ejemplo 5K ó 10K) en el que no hay información con respecto al orden en el que aparecen las palabras (de ahí lo de _bag_).

Pero hay muchas otras tareas PLN en las que no podemos despreciar ningún elemento del texto. Son tareas en las que lo que interesa no es descubrir _de qué va el texto_ (como en la clasificación de documentos) sino _qué relación existe entre las palabas del texto_ (como, por ejemplo, en la extracción de información). Para estas tareas el modelo BOW se queda claramente corto y se aplican procesamientos que _enriquecen los textos_ y no los simplifican (como ocurre en el caso del BOW).

Uno de esos procesos de enriquecimiento es el etiquetado POS (_Part Of Speech_), que consiste en determinar la categoría gramatical de cada palabra del texto. <code>NLTK</code> dispone de etiquetadores POS _pre-entrenados_ y también nos da la posibilidad de entrenar nuvos etiquetadores a partir de nuestros propios corpus anotados.

Lo primero que necesitamos para poder aplicar un etiquetador POS es convertir una frase en una lista de palabras. Usaremos la misma función <code>word_tokenize</code> que ya hemos usado anteriormente:

In [ ]:
# La famosa frase inicial de El Quijote ocupa la posición 4 en nuestra lista de frases
frase_inicial = frases[4]
print(frase_inicial, '\n')

# La convertimos en lista de palabras que es la entrada que espera el etiquetador POS
palabras_frase_inicial = nltk.word_tokenize(frase_inicial, language='spanish')
print(palabras_frase_inicial)

A partir de una lista de palabras, simplemente llamando a la función <code>pos_tag</code> conseguimos asignar una categoría gramatical a cada palabra. En la siguiente celda vemos cómo hacerlo y cómo mostrar el resultado:

In [ ]:
# Etiquetar con el tagger por defecto es así de simple
pos_frase_inicial = nltk.pos_tag(palabras_frase_inicial)

# Se genera una lista de tuplas (palabra, pos)
print(pos_frase_inicial)

Como hemos visto en la celda anterior, es muy fácil obtener las categorías gramaticales de una palabra. Pero si nos fijamos un poco, vemos que la salida es bastante mala. Solo con la primera palabra vemos que la cosa no va bien ya que 'En' se etiqueta como nombre (<code>NN</code>) y le siguen muchos errores de bulto más. La razón de esta baja calidad es que estamos usando un etiquetador _por defecto_, entrenado para inglés sobre el corpus _Penn Treebank_.

Para poder etiquetar textos en otros idiomas, necesitamos un corpus anotado y entrenar un etiquetador sobre ese corpus. <code>NLTK</code> nos da soluciones para ambas cosas. En primer lugar, viene con una colección de recursos anotados para varios idiomas. En la siguiente celda, por ejemplo, vemos cómo podemos descargarnos el corpus _CESS-ESP_ a través de <code>NLTK</code>:

In [ ]:
# Descargamos el corpus CESS-ESP a través de NLTK
nltk.download('cess_esp')

# Probamos a mostrar la primera frase del corpus
print(nltk.corpus.cess_esp.tagged_sents()[0])

Una vez que tenemos el corpus, entrenar el etiquedador es bastante sencillo. Solo tenenos que seguir estos cuatro pasos:
- Leer el corpus y guardarlo en una variable
- Determinar qué etiqueta asociar _por defecto_ a las palabras desconocidas
- Crear un objeto _etiquetador_ (nostotros hemos escogido el etiquetador <code>TnT</code> pero hay más disponibles)
- Entrerar el etiquetador usando el corpus

In [ ]:
# Cargamos el corpus de entrenamiento
corpus_entrenamiento = nltk.corpus.cess_esp.tagged_sents()

# Establecemos la eqtiqueta a asignar a las palabras desconocidas
pos_palabra_desconocida = nltk.tag.DefaultTagger('ncms000')

# Creamos el objeto 'tnt_tagger', con la etiqueta de las palabras desconocidas y con la capacidad de ser entrenado
tnt_tagger = nltk.tag.tnt.TnT(unk=pos_palabra_desconocida, Trained=True)

# Entrenamos el etiquetador
tnt_tagger.train(corpus_entrenamiento)

Ya tenemos el etiquetador entrenado en el objeto <code>tnt_tagger</code>, que nos ofrece el método <code>tag</code> que etiqueta cualquier frase que le pasemos en formato lista de palabras:

In [ ]:
# Usamos el método 'tag' del objeto 'tnt_tagger'
pos_frase_inicial = tnt_tagger.tag(palabras_frase_inicial)

# Y ahora las etiquetas son las del corpus CESS-ESP
print(pos_frase_inicial)

## 4. WordNet <a name="sec_wordnet"/> 

<code>WordNet</code> es una base de datos léxica para el inglés, creada en la Universidad de Princeton, que agrupa palabras en conjuntos de sinónimos. La filosofía de diseño de WordNet se apoya en una idea similar a la del conocido refrán _"dime con quién andas y te diré quién eres"_. Una versión _semántica_ de este refrán podría ser _"dime a qué palabras se parece y te diré lo que significa"_. Esto es precisamente lo que hace <code>WordNet</code>, agrupa palabra en conjuntos de palabras sinónimas (_synsets_) y simplemente con eso se puede disponer de un modelo semántico de las palabras de un idioma. Cada _synset_ es una acepción, y las palabras participan en tantos _synsets_ como significados tengan.

Además de la sinonimia, que es la relación principal, <code>WordNet</code> incluye otras relaciones semánticas entre palabras muy útiles como la hipernonimia, hiponimia, holonimia o meronimia. De ahí el nombre del recurso, <code>WordNet</code>, que hace referencia a una red de palabras.

Desde su creación en 1985, <code>WordNet</code> se ha convertido en el recurso de referencia en la tarea de representar el significado de las palabras. Desde entonces, han aparecido multitud de proyectos para relacionar <code>WordNet</code> con otros recursos: ontologías, y en especial léxicos de otros idiomas. La asociación _Global WordNet Association_ proporciona una plataforma para discutir, compartir y conectar todos estos recursos en torno a <code>WordNet</code>. En este _notebook_ trabajaremos con la versión original para el inglés y también con la interfaz de _Open Multilingual WordNet_, un proyecto integrado en <code>NLTK</code> que ofrece una interfaz para más de 30 idiomas, entre ellos el español.

Para empezar a usar <code>WordNet</code> desde <code>NLTK</code>, lo primero que tenemos que hacer es importarlo:

In [ ]:
from nltk.corpus import wordnet as wn

Con la función <code>synsets</code> podemos recuperar todos los _synsets_ en los que dicha palabra participa:

In [ ]:
synsets_bank = wn.synsets('bank')              # Obtención de todos los synsets a partir de una palabra
print(type(synsets_bank[0]))                   # Mostramos el tipo de un synset
print(synsets_bank)                            # Mostramos la lista de synsets

Y podemos acceder a la definicón en forma de glosa de cada _synset_ con el método <code>definition</code>:

In [ ]:
# Descripción de todos los synsets
for synset in synsets_bank:                       # Para todos los synsets en la lista 'synsets_bank'
    print(synset, synset.definition())            # El método 'definition' nos devuelve la definición en forma de glosa

Además de la definición, también podemos acceder a más informaciones a partir de un objeto tipo <code>Synset</code>. En concreto, dos de las informaciones más útiles serán los lemas asociados a un _synset_ y ejemplos de uso en forma de frases:

In [ ]:
# Informaciones asociadas a un synset

# Segundo synset de la lista asociada a 'bank'. Su significado es entidad bancaria
entidad_bancaria = synsets_bank[1]

# Lemas asociados a ese synset (palabras sinónimas)
print(entidad_bancaria.lemmas())

# Ejemplos de uso del lema 'bank' con este significado
print(entidad_bancaria.examples())

La interfaz que ofrece <code>NLTK</code> para acceder a <code>WordNet</code> está preparada para trabajar con distintos idiomas. Para poder hacerlo, es necesario antes descargar el recurso <code>omw</code> con la función <code>download</code> que ya hemos usado anteriormente para descargar otros recursos integrados en <code>NLTK</code>:

In [ ]:
# Descarga de Open Multilingual WordNet
nltk.download('omw')

Ahora simplememte tendremos que dar el valor apropiado al parámetro <code>lang</code> disponible en muchas de las funciones que nos proporciona <code>NLTK</code> para indicar el idioma con el que queremos trabajar. Por ejemplo:

In [ ]:
# Cáculo de los synsets correspondientes a la palabra 'banco' en español
synsets_banco = wn.synsets('banco', lang='spa')

for synset in synsets_banco:
    print(synset, synset.definition())

Las definiciones siguen estando en inglés. Esto se debe a que no estamos usando <code>WordNet</code> para el español, sino que etamos conectando el léxico del español con la verisón original de <code>WordNet</code> para el inglés. Esto tiene la doble ventaja de aprovechar todas las relaciones disponibles en <code>WordNet</code>, al tiempo que abre la puerta a la posibilidad de realizar aplicaciones multilingues de forma muy natural.

En la siguente celda vamos a dar un pequeño paso más en el _curso implícito de programación_ que estamos siguiendo con este _notebook_. Se trata de definir una función. Hasta ahora, hemos usado muchos métodos y funciones que ya estaban disponibles (bien desde <code>Python</code> o desde <code>NLTK</code>). Pero con los lenguajes de programación podemos definir nuestras propias funciones. Al definir una función, básicamente lo que hacemos es _encapsular_ un trozo de programa para poder utilizarlo muchas veces sin tener que repetirlo. Una definición de función tiene dos elememtos fundamentales:
- la cabecera: en la que se establece el nombre y los parámetros
- el cuerpo: con las instrucciones que se ejecutarán cada vez que esa función _es llamada_

La siguiente función se apoya en el método <code>lemmas</code> de los objetos <code>Synset</code> para calcular los sinónimos de una palabra. Tiene, además, un parámetro que nos permite indicarle en qué idioma queremos buscar los sinónimos:

In [ ]:
def calcula_sinonimos(palabra, idioma='eng'):
    ''' Función que calcula las palabras sinónimas de una palabra dada
    '''
    sinonimos = set()                                # Conjunto de sinonimos inicialmente vacío
    for syn in wn.synsets(palabra, lang=idioma):     # Para cada synset en el que aparezca la palabra
        for lema in syn.lemmas(lang=idioma):         # Para cada lema de cada synset
            sinonimos.add(lema.name())               # Se añade el lema (en el atributo 'name') al conjunto
    return sinonimos                                 # Se devuelve el conjunto de sinónimos

Con las siguinetes dos celdas podemos comprobar cómo funciona nuestra primera función, mostrando los sinónimos de las palabras _good_ y _bueno_:

In [ ]:
# Cálculo de los sinónimos de la palabra 'good'
sinonimos_good = calcula_sinonimos('good')           # El idioma por defecto es 'eng' y no es necesario indicarlo
print(sinonimos_good)

In [ ]:
# Cálculo de los sinónimos de la palabra 'bueno'
sinonimos_bueno = calcula_sinonimos('bueno', idioma='spa')
print(sinonimos_bueno)

Como ya hemos comentado, <code>WordNet</code> incluye más relaciones además de la que es su relación principal, la sinonimia. Las más importantes son la hiperonimia, hiponimia, holonimia y meronimia. En la siguiente celda vemos ejemplos de las funciones que nos ofrece <code>NLTK</code> para utilizar estas relaciones:

In [ ]:
# Otras relaciones entre synsets
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')

# Hiperónimos (inmediatos) de 'dog'
print(dog.hypernyms())

# Hipónimos (inmediatos) de 'dog'
print(dog.hyponyms())

# Hiperónimo ráiz de 'dog'
print(dog.root_hypernyms())

# Hiperónimo común más cercano de 'dog' y 'cat'
print(dog.lowest_common_hypernyms(cat))

La red de palabras de <code>WordNet</code> permite hacer más cosas además de recuperar palabras mediante distintos tipos de relaciones. En una red, los nodos (en nuestro caso los _synsets_) están conectados por arcos (las relaciones), y gracias a esa conexión se puede determinar el grado de cercanía entre nodos:
- Si dos nodos están conectados por un arco serán _muy cercanos_
- Si hay que dar dos (o más) _saltos_  para llegar de un nodo a otro, estos nodos estarán _más alejados_

A este tipo de relación entre nodos la denominaremos _distancia_ y es un instrumento muy potente para comparar palabras. <code>NLTK</code> proporciona varias formas de calcular distancias entre dos _synsets_ aprovechando la red de relaciones de <code>WordNet</code>. En la siguiente celda tenemos tres de ellas:

In [ ]:
# Algunas formas de calcular similitudes entre dos synsets
bank = wn.synset('bank.n.01')
beach = wn.synset('beach.n.01')

# Basada en el camino más corto que une los synsets (0-> similitud mínima, 1-> similitud máxima)
print(bank.path_similarity(beach))

# Leacock-Chodorow Similarity: basada en el camino más corto y en la profundidad taxonómica de los synsets
print(bank.lch_similarity(beach))

# Wu-Palmer Similarity: basada en la profundidad taxonómica de los synsets y el ancestro común más cercano 
print(bank.wup_similarity(beach))

En la siguiente celda vemos el resultado de aplicar una de estas métricas (<code>wup_similarity</code>) comparando el _synset_ correspondiente a _banco fluvial_ con otros tres _synsets_ que tienen distintos grados de parecido con él:

In [ ]:
# Similitud entre banco fluvial y playa
bank = wn.synset('bank.n.01')
beach = wn.synset('beach.n.01')
print(bank.wup_similarity(beach))

# Similitud entre banco fluvial y montaña
mountain = wn.synset('mountain.n.01')
print(bank.wup_similarity(mountain))

# Similitud entre banco fluvial y gato
cat = wn.synset('cat.n.01')
print(bank.wup_similarity(cat))

## 5. Word2vec (gensim) <a name="sec_word2vec"/> 

<code>Word2vec</code> es un conjunto de técnicas que nos permiten construir modelos de _word embeddings_. Estas técnicas están basadas en redes neuronales de dos capas que se entrenan para reconstruir contextos de palabras (una idea cercana a la de los _autoencoders_ usados en ciertas técnicas de _deep learning_). 

Los _word embeddings_ son representaciones de palabras mediante vectores en un espacio continuo. Eso de _espacio continuo_ significa que se usan números reales en lugar de números naturales en los vectores, y viene a poner de manifiesto la diferencia con los modelos BOW (que en última instancia se apoyan en la frecuencia de las palabras, que son números naturales). 

La otra gran diferencia con los modelos BOW es el número de dimensiones. Mientras que los vectores BOW tienen tantas dimensiones como el tamaño del vocabulario (varios miles), los _word embeddings_ requieren solo de unas pocas centenas de dimensiones.

Una de las mayores utilidades de los _word embeddings_ es su capacidad para capturar el significado de las palabras. El modelo de representación es radicalmente distinto al de <code>WordNet</code> y además se puede construir de forma no supervisada a partir de grandes colecciones de textos sin anotar (miles de millones de palabras).

En este _notebook_ usaremos los _word embeddings_ a través de una de las implementaciones que más popular ha hecho su uso: <code>gensim</code>. Esta herramienta no solo permite trabajar con _word embeddings_ sino que incluye, además, muchas otras técnicas de representación vectorial de palabras y documentos (TF-IDF, LDA, LSA, ...). <code>gensim</code> está en la distribución de <code>Anaconda</code> pero no es uno de los paquetes que se instala por defecto cuando instalamos <code>Anaconda</code>. Para instalarlo debemos ejecutar <code>conda install gensim</code> desde una ventana de _Anaconda Prompt_.

Como siempre, empezaremos por importar el paquete:

In [ ]:
import gensim

Para empezar a trabajar con <code>gensim</code> necesitamos un modelo. Podemos entrenar uno, para lo que necesitaremos muchos textos (se puede trabajar con un _dump_ de la Wikipedia que son fáciles de encontrar en Internet). O trabajar con un modelo ya pre-entrenado. Nosotros usaremos el modelo del _Spanish Billion Corpus_ disponible en http://crscardellino.me/SBWCE/. Una vez descargado, lo copiaremos en la carpeta <code>c:\gensim-models\</code>

En la siguiente celda se muestra como cargar en memoria el modelo. Los modelos pueden estar en formato binario (es el formato del _Spanish Billion Corpus_) o en formato texto. En la celda aparecen un par de ejemplos (comentados, para que no se ejecuten) que muestran cómo cargar un modelo en formato texto.

Los modelos suelen ser bastante voluminosos (1Gb o más), por lo que esta operación puede llevar algo de tiempo. Si el ordenador no tiene capacidad suficiente de memoria, estas instrucciones pueden provocar incluso un error. En esos casos la alternativa sería entrenar un modelo más pequeño (y de menor calidad) para poder cargarlo en memoria.

In [ ]:
# Carga del modelo (puede tardar uno o dos minutos, dependiendo del ordenador)

# Carga del modelo Spanish-Billion-Corpus (300 dimensiones, formato binario)
modelo = gensim.models.KeyedVectors.load_word2vec_format('c:/gensim-models/SBW-vectors-300-min5.bin.gz', binary=True)

# Carga de un modelo reducido (200Mb, 150 dimensiones, formato binario)
#modelo = gensim.models.KeyedVectors.load_word2vec_format('c:/gensim-models/wiki-es-150.bin', binary=True)

Ya podemos empezar a trabajar con el modelo. En la siguiente celda se muestra cómo obtener el tamaño de los vectores de un modelo, las primeras componentes de un vector y algunas estadísticas sobre los valores de las componentes de un vector:

In [ ]:
# Número de componentes de los vectores de un modelo 
print(modelo.vector_size)

# Acceso al vector de una palabra
vector_hombre = modelo['hombre']

# Las primeras 10 componentes del vector
print(vector_hombre[:10])

# Valor máximo de las componentes
print(max(vector_hombre))

# Valor mínimo de las componentes
print(min(vector_hombre))

# Valor medio de las componentes
print(sum(vector_hombre)/len(vector_hombre))

Cuando una palabra no está en el modelo e intentamos recuperar su vector, tendremos como resultado un error: 

In [ ]:
print(modelo['abraza-farolas'])

La función más importante que nos ofrece <code>gensim</code> es <code>similarity</code> que nos permite calcular el grado de similitud entre dos palabras en función de sus respectivos vectores. 

En la siguiente celda vemos el resultado de aplicar estas función comparando el vector de la palabra _hombre_ con los vectores de otras tres palabras que tienen distintos grados de parecido con ella:

In [ ]:
# Similitud entre 'hombre' y 'persona'
print(modelo.similarity('hombre', 'persona'))

# Similitud entre 'hombre' y 'mono'
print(modelo.similarity('hombre', 'mono'))

# Similitud entre 'hombre' y 'metal'
print(modelo.similarity('hombre', 'metal'))

<code>gensim</code> incluye algunas operaciones curiosas que se apoyan en vectores de palabras. La siguiente celda muestra una de ellas, la búsqueda de la palabra que más _desentona_ dentro de una lista de palabras:

In [ ]:
# Determinar la palabra que no cuadra en una lista
print(modelo.doesnt_match(['bicicleta', 'moto', 'coche', 'melocotón']))

Otra de estas operaciones predefinidas es <code>most_similar</code>, que calcula la palabra más similar dados dos conjuntos de palabras que determinan, respectivamente, un polo positivo y otro negativo. Esta función es la que permite dar respaldo a la famosa _fórmula algebraica_ que aparece en muchas introducciones a los _word embeddings_, y que reza que 
$$
reina \; = \; rey \; - \; hombre \; + \; mujer
$$
En la siguiente celda vemos el resultado de usar esta función: 

In [ ]:
# Determinar las palabras más:
#      - parecidas a un conjunto de palabras (polo positivo), y
#      - diferentes a un conjunto de palabras (polo negativo)
print(modelo.most_similar(positive=['mujer', 'rey'], negative=['hombre']))

Pero, sin duda, el uso más interesante que podemos hacer de los vectores que nos proporciona <code>gensim</code> parte del hecho de que podemos aplicar cualquier operación numérica. Lo más común es utilizar funciones de distancia para comparar los vectores de dos palabras. Hay muchas distancias definidas para vectores, una de las más populares es la similitd del coseno. Está basada en el producto escalar de vectores y toma valores entre $1$ (similitud máxima) y $-1$ (distancia máxima). Se calcula con la siguiente fórmula:

$$
similitud\_coseno(A,B) = \frac{\sum{a_ib_i}}{\sqrt{\sum{a_i^2}}\sqrt{\sum{b_i^2}}}
$$

La similitud del coseno no está definida para vectores con todas sus componentes a cero, ya que nos llevará a una división por cero. Tendremos que tener en cuenta este aspecto en nuestros experimentos para evitar ese tipo de situaciones.

La siguiente celda incluye la implementación en <code>Python</code> de la similitud del coseno. Inlcuye una pequeña modificación de la fórmula que permite vectores con ceros en todos sus componentes. En esos casos se devolverá la similitud $0$, que está a mitad de camino entre el mínimo $-1$ y el máximo $1$:

In [ ]:
from math import sqrt

def similitud_coseno(vector_a, vector_b):
    ''' Calcula la similitud del coseno de dos vectores
    '''
    sum_abs_a = sum([abs(a) for a in vector_a])                 # Suma de los valores absolutos de las as
    sum_abs_b = sum([abs(b) for b in vector_b])                 # Suma de los valores absolutos de las bs
    if sum_abs_a == 0 or sum_abs_b==0:                          # Si algunos de los vectores tiene todos los valores ceros 
        return 0                                                # Devolvemos la similitud 0
    else:                                                       # Si ninguno de los vectores es nulo
        aa = sum([a*a for a in vector_a])                       # Suma de los cuadrados de las as
        bb = sum([b*b for b in vector_b])                       # Suma de los cuadrados de las bs
        ab = sum([a*b for a, b in zip(vector_a, vector_b)])     # Suma de los productos ab
        return (ab/(sqrt(aa)*sqrt(bb)))                         # Cálculo de la similitud

Ya podemos usar nuestra implementación de la similitud del coseno sobre los vectores de <code>Word3vec</code>:

In [ ]:
# Podemos usar los vectores para calcular
vector_cielo = modelo['cielo']
vector_nube = modelo['nube']
vector_angulo = modelo['angulo']

print(similitud_coseno(vector_cielo, vector_nube))
print(similitud_coseno(vector_cielo, vector_angulo))

## 6. Similitud de textos <a name="sec_similitud"/> 

En esta última sección usaremos varios de los elementos vistos en el resto del _notebook_ para realizar un experimento simple de similitud. Trabajaremos con una colección de pares de textos extraidos de la competición _Semantic Text Similarity (STS)_ de SemEval-2015. Esta colección, y todos los recursos de las distintas ediciones de _STS_, se pueden encontrar en su wiki: http://ixa2.si.ehu.es/stswiki/index.php/Main_Page

Los datos vienen organizados en dos ficheros:
- <code>STS.input.wikipedia.txt</code>: con las parejas de frases separadas por tabuladores (<code>'\t'</code>)
- <code>STS.gs.wikipedia.txt</code>: con el grado de similitud (_gold standard_) de cada pareja

El objetivo del experimento es calcular un grado de similitud para cada pareja haciendo uso de los vectores que nos proporciona <code>Word2vec</code>. Evaluaremos nuestro experimento comparando estas estimaciones de similitud con los valores de similitud proporicionados en el _gold standard_. Para ello haremos uso del coeficiente de correlación de Pearson, tal y como se hizo en la competición de SemEval-2015.

Lo primero que necesitamos implementar, son las funciones para leer los datos de entrada. La función <code>lee_fichero_frases</code> leerá el contenido del fichero de pares de frases y lo guardará en una lista de tuplas. Por su parte la función <code>lee_fichero_puntuaciones</code> leerá el contenido del fichero de puntuaciones y lo guardará en una lista de números:

In [ ]:
def lee_fichero_frases(fichero):
    ''' Lee un fichero de frases y construye una lista de tuplas (frase1, frase2)
    '''
    f = open(fichero, encoding='utf-8')                # Apertura del fichero
    parejas = []                                       # Lista de parajas de frases inicialmente vacía
    for linea in f:                                    # Para cada línea del fichero de entrada
        frase1, frase2 = linea.strip().split('\t')     # Quitamos fin de línea y separamos por '\t'
        parejas.append((frase1, frase2))               # Añadimos la pareja de frases a la lista
    f.close()                                          # Cierre del fichero
    return(parejas)                                    # Devolvemos la lista de parejas de frases

def lee_fichero_puntuaciones(fichero):
    ''' Lee un fichero de puntuaciones y construye una lista de números
    '''
    f = open(fichero)                                  # Apertura del fichero
    puntuaciones = []                                  # Lista de puntuaciones inicialmente vacía
    for linea in f:                                    # Para cada línea del fichero de entrada
        puntuacion = float(linea)                      # Convertimos a número real
        puntuaciones.append(puntuacion)                # Añadimos la puntuación a la lista
    f.close()                                          # Cierre del fichero
    return(puntuaciones)                               # Devolvemos la lista de puntuaciones

La siguiente celda contiene unas pruebas de ambas funciones:

In [ ]:
# Lectura de la lista de parejas de frases
parejas = lee_fichero_frases('./datos/STS.input.wikipedia.txt')

# Lectura de la lista de puntuaciones
puntuaciones = lee_fichero_puntuaciones('./datos/STS.gs.wikipedia.txt')

# Tests: mostrar el número de elementos y los dos primeras elementos
print(len(parejas), parejas[:2])
print(len(puntuaciones), puntuaciones[:2])

La siguiente función realiza un preprocesado básico con <code>NLTK</code> para tokenizar las frases y eliminar palabras irrelevantes:

In [ ]:
def calcula_tokens_relevantes(frase, idioma='spanish'):
    ''' Convierte una frase en una lista de tokens y elimina tokens irrelevantes 
    '''
    # Tokenización
    tokens = nltk.word_tokenize(frase, language=idioma)
    
    # Eliminación de símbolos no alfabéticos (de puntuación)
    tokens = [t for t in tokens if t.isalpha()]

    # Conversión a minúsculas
    tokens = [t.lower() for t in tokens]

    # Eliminación de stopwords
    stopwords = nltk.corpus.stopwords.words(idioma)
    tokens = [t for t in tokens if t not in stopwords]
    
    # Convertimos la lista en conjunto para eliminar duplicados
    tokens = set(tokens)
    
    # Volvemos a convertirla a lista para mantener un orden
    return list(tokens)

In [ ]:
# Test de la función tokens relevantes
frase_eugenia_cheng = 'Las matemáticas son el estudio lógico de cómo funcionan las cosas lógicas.'
print(calcula_tokens_relevantes(frase_eugenia_cheng))

Como no estamos seguros de que todas las palabras que tengamos que procesar están en nuestro modelo, vamos a contemplar esa situación en la función <code>vector</code> que se comportará de la siguiente forma:
- devolverá el vector de la palabra si la palabra está en el modelo, y 
- devolverá el valor nulo <code>None</code> si la palabra no está en el modelo

In [ ]:
def vector(palabra, modelo):
    ''' Calcula el vector de una palabra según un modelo
    
    Si la palabra no está en el modelo, se devuelve el valor None
    '''
    try:                                            # Intentamos recuperar el vector
        resultado = modelo[palabra]                 # Si existe lo guardamos en la variable resultado
    except KeyError:                                # Si no existe y se provoca un error
        resultado = None                            #    Guardamos el valor None en resultado
    return resultado                                # Devolvemos el resultado 

La siguiente función calcula el vector correspondiente a una secuencia de palabras a partir de los vectores de las palabras. Utilizaremos una estrategia simple, consistente en calcular la media de los vectores de las palabras.
Haremos uso de la función <code>zeros</code> del paquete <code>Numpy</code> para generar un vector inicial de ceros sobre el que iremos sumando todos los vectores para calcular, en última instancia, la media:

In [ ]:
import numpy as np

def calcula_vector_palabras(palabras, modelo):
    ''' Calcula la media de los vectores de una lista de palabras 
    '''
    vector_suma = np.zeros(modelo.vector_size)                     # Vector de ceros sobre el que sumaremos
    num_palabras = 0                                               # Contaremos el número de palabras con vector
    for palabra in palabras:                                       # Para todas las palabras de la lista
        v = vector(palabra, modelo)                                # Calculamos el vector
        if v is not None:                                          # Si el vector no es nulo
            vector_suma = vector_suma + v                          #    Sumamos el vector
            num_palabras = num_palabras + 1                        #    Contamos una palabra más
    if num_palabras>0:                                             # Si hay al menos una palabra con vector
        return vector_suma / num_palabras                          #    Devolvemos la media
    else:                                                          # Si no hay ninguna palabra con vector
        return np.zeros(modelo.vector_size)                        #    Devolvemos un vector con todo ceros         

In [ ]:
# Test de la función calcula_vector_palabras
print(calcula_vector_palabras(['lunes', 'al', 'sol'], modelo))

Con las funciones que tenemos ya implementadas, el cálculo de la similitud de dos frases es bastante simple. Solo necesitamos aplicar la función de preprocesamiento, calcular el vector correspondiente a una secuencia de palabras con la función <code>calcula_vector_palabras</code>, y por último aplicar la función de similitud:

In [ ]:
def calcula_similitud_frases(frase1, frase2, modelo, idioma='spanish'):
    ''' Calcula la similitud de dos frases según un modelo
    
    Usaremos la distancia del coseno (0-> más parecido, 1-> menos parecido)
    1-distancia nos da la similitud (1-> más parecido, 0-> menos parecido)
    '''
    tokens1 = calcula_tokens_relevantes(frase1, idioma)  # Calcula los tokens de la frase 1
    tokens2 = calcula_tokens_relevantes(frase2, idioma)  # Calcula los tokens de la frase 2
    vector1 = calcula_vector_palabras(tokens1, modelo)   # Calcula el vector de la frase 1
    vector2 = calcula_vector_palabras(tokens2, modelo)   # Calcula el vector de la frase 2
    return(similitud_coseno(vector1, vector2))           # Devuelve la similiud del coseno

In [ ]:
# Test de la función calcula_similitud_frases
frase_eugenia_cheng = 'Las matemáticas son el estudio lógico de cómo funcionan las cosas lógicas.'
frase_martin_gardner = 'Un matemático es una máquina de transformar café en teoremas'
print(calcula_tokens_relevantes(frase_eugenia_cheng))
print(calcula_tokens_relevantes(frase_martin_gardner))

print(calcula_similitud_frases(frase_eugenia_cheng, frase_martin_gardner, modelo))

Ya estamos muy cerca de nuestro obetivo final, tenemos una función que nos calcula un valor entre $-1$ y $1$ dadas dos frases, y lo único que nos falta es adaptar ese valor al rango de valores definido en la competición (entre $0$ y $4$). Lo haremos a través de la función <code>calcula_puntuacion</code> aplicando una estrategia muy simple, sumar $1$ y multiplicar por $2$:

In [ ]:
def calcula_puntuacion(similitud):
    ''' Convierte una similitud (entre -1 y 1) en una puntuación STS-SemEval (entre 0 y 4)
    
    Usaremos una aproximación muy simple que consiste en sumar 1 y multiplicar por 2.
    
    En este aspecto hay bastante margen de mejora. Por ejemplo, usar machine learning para
    aprender cuales son los umbrales óptimos de similitud para cada nivel de la tarea STS.
    '''
    return 2*(similitud+1)

En la última funcion <code>experimento</code> pondremos todas las piezas juntas para calcular la lista de puntuaciones estimadas con nuestra técnica de cálculo de similitud, y compararlas con el _gold standard_ para obtener el coeficiente de correlación de Pearson:

In [ ]:
from scipy.stats import pearsonr

def experimento(parejas, puntuaciones, modelo, idioma='spanish'):
    '''  Estima las puntuaciones de una lista de parejas de frases y las compara con las puntuaciones reales
    
    Al igual que en las competiciones STS-SemEval, se usa el coeficiente de correlación de Pearson
    para comparar la lista de puntuaciones estimada con la lista de puntuaciones reales. Usaremos la
    implementación de scipy.stats que produce dos valores:
       - coeficiente de correlación: es el que nos interesa. Está entre 1 (mejor correñación) y -1 (correlación negativa)
       - pvalue: en cierto modo, mide la fiabilidad del coeficiente (en la evaluación de STS-SemEval no se tuvo en cuenta)
    '''
    puntuaciones_estimadas = []                                         # Lista vacía de puntuaciones estimadas         
    for frase1, frase2 in parejas:                                      # Para todos los pares de frases
        sim = calcula_similitud_frases(frase1, frase2, modelo, idioma)  # Calculamos la similitud
        estimacion = calcula_puntuacion(sim)                            # Calculamos la puntuación a partir de la similitud
        puntuaciones_estimadas.append(estimacion)                       # Añadimos la estimación a la lista
    correlacion, _ = pearsonr(puntuaciones, puntuaciones_estimadas)     # Obtenemos la correlacion (ignoramos el p-value)
    return((correlacion, puntuaciones_estimadas))                       # Devolvemos la correlación y las puntuaciones estimadas

Y, por último, llamamos a la función <code>experimento</code> para obtener el resultado final de nuestra prueba. Calcularemos, también, otros indicadores que puedan darnos pistas para saber por dónde seguir mejorando el rendimiento de nuestro sistema:

In [ ]:
# Resultado del experimento
correlacion, puntuaciones_estimadas = experimento(parejas, puntuaciones, modelo)
print("El coeficiente de correlación es:", correlacion)

# Algunas estadístcas de las puntuacines reales y estimadas
print("\nPuntuaciones máximas:")
print("   - real:", max(puntuaciones))
print("   - estimada:", max(puntuaciones_estimadas))

print("\nPuntuaciones mínimas:")
print("   - real:", min(puntuaciones))
print("   - estimada:", min(puntuaciones_estimadas))

print("\nPuntuaciones medias:")
print("   - real:", sum(puntuaciones)/len(puntuaciones))
print("   - estimada:", sum(puntuaciones_estimadas)/len(puntuaciones_estimadas))

A partir de este esquema de solución se pueden empezar a probar variantes para intentar mejorar los resultados. Un par de ideas para empezar son:
- El efecto de no eliminar _stopwords_, o reducir la lista de _stopwords_ para eliminar menos palabras. Puede que alguna de las palabras eliminadas contengan información relevante para el cálculo de la similitud.
- Experimentar con distintos umbrales para trasladar el rango de la similitud del coseno (de $-1$ a $1$) al rango de puntos definido por la competición (de $0$ a $4$). La diferencia observada entre los valores mínimos, máximos y medios de las puntuaciones reales y estimadas, muestran que la conversión de rango implementada en la función <code>calcula_puntuacion</code> es claramente mejorable.